In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.prebuilt import create_react_agent
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Dict
from config import Settings
import os

In [ ]:
api_key = Settings.api_key
VECTOR_DIR = 'vectorize'
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=api_key)